* Build and fit a model using the same data we have here but with the MobileNetV2 architecture feature extraction (mobilenet_v2_100_224/feature_vector) from TensorFlow Hub, how does it perform compared to our other models?
* Name 3 different image classification models on TensorFlow Hub that we haven't used.
* Build a model to classify images of two different things you've taken photos of.
* You can use any feature extraction layer from TensorFlow Hub you like for this.
* You should aim to have at least 10 images of each class, for example to build a fridge versus oven classifier, you'll want 10 images of fridges and 10 images of ovens.

In [3]:
import tensorflow as tf
import tf_keras
import tensorflow_hub as hub
import tensorflow.keras.layers as layers

In [1]:
def create_model(model_url, num_classes=10):

  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """

  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, input_shape=IMAGE_SHAPE+ (3,), name='feature_extraction_layer')

  #Create model (add layers after creation of model or else errors will occur)
  model = tf_keras.Sequential()
  model.add(feature_extractor_layer)
  model.add(tf_keras.layers.Dense(num_classes, activation=tf.keras.activations.softmax, name='output_layer'))

  return model

In [5]:
model_url = "https://kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/1"
IMAGE_SHAPE = (224, 224)
model = create_model(model_url, num_classes=10)

In [6]:
#Compile our resnet model
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = "Adam",
    metrics = ["accuracy"])

In [7]:
# Get data (10% of 10 food classes from Food 101)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip it
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2025-05-27 19:32:35--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 142.251.188.207, 74.125.142.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   158MB/s    in 1.0s    

2025-05-27 19:32:36 (158 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [10]:
# Setup data inputs
train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

In [11]:
# Set up datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale = 1/255.,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   rotation_range = 0.2,
                                   zoom_range = 0.2,
                                   shear_range = 0.2,
                                   )
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training images:")
# Create image data generator
train_data_aug = train_datagen.flow_from_directory(
    train_dir,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    target_size = IMAGE_SHAPE
)
train_data = test_datagen.flow_from_directory(
    test_dir,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    target_size = IMAGE_SHAPE
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    target_size = IMAGE_SHAPE
)


Training images:
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [12]:
from logging import log
import tensorflow as tf
import tf_keras

# Create TensorBoard callback (functionized because we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf_keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving Tensorflow log files to : {log_dir}")
  return tensorboard_callback

In [14]:
history = model.fit(train_data,
                               epochs = 10,
                               steps_per_epoch = len(train_data),
                               validation_data = test_data,
                               validation_steps = len(test_data),
                               callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                                                        experiment_name = "resnet50v2")]
                               )

Saving Tensorflow log files to : tensorflow_hub/resnet50v2/20250527-193924
Epoch 1/10
 7/79 [=>............................] - ETA: 1:36 - loss: 2.3908 - accuracy: 0.1429

KeyboardInterrupt: 

* InceptionV3:
* Description: InceptionV3 is a convolutional neural network architecture that is part of the Inception family of models. It focuses on making networks deeper and wider while keeping the computational budget in check. It uses inception modules, which perform convolutions with different filter sizes and pooling operations in parallel.
* Important Stuff: It is known for its good performance on image classification tasks and was a popular choice before the advent of more recent architectures. It is a relatively large model compared to MobileNet.
NASNet-A:
* Description: NASNet (Neural Architecture Search Network) models are developed using neural architecture search, which is a technique to automatically design neural network architectures. NASNet-A refers to a specific architecture found using this method.
* Important Stuff: These models are known for achieving state-of-the-art results on image classification benchmarks. They can be computationally more expensive than hand-designed architectures.
PNASNet-5Large:

* Description: PNASNet (Progressive Neural Architecture Search Network) is another architecture developed using neural architecture search.
* PNASNet-5Large is a larger variant of this architecture.
* Important Stuff: Similar to NASNet, PNASNet models also achieve high accuracy on image classification tasks. They are generally powerful but can require significant computational resources.